# VectorStoreRetrieverMemory


`VectorStoreRetrieverMemory` 는 벡터 스토어에 메모리를 저장하고 호출될 때마다 가장 '눈에 띄는' 상위 K개의 문서를 쿼리합니다.

이는 대화내용의 순서를 **명시적으로 추적하지 않는다는 점** 에서 다른 대부분의 메모리 클래스와 다릅니다.


In [8]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

먼저, 벡터 스토어를 초기화 합니다.


In [9]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


# 임베딩 모델을 정의합니다.
# 비용이 나가지만 크지는 않음
embeddings_model = OpenAIEmbeddings()

# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

실제 사용에서는 `k`를 더 높은 값으로 설정하지만, 여기서는 `k=1` 을 사용하여 다음과 같이 표시합니다.


In [10]:
from langchain.memory import VectorStoreRetrieverMemory

# 벡터 조회가 여전히 의미적으로 관련성 있는 정보를 반환한다는 것을 보여주기 위해서입니다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
memory = VectorStoreRetrieverMemory(retriever=retriever)

# 임의의 대화를 저장합니다.
memory.save_context(
    inputs={"human": "지혜야, 이번 주 토요일(8월 23일)에 난 홍대에서 인디 밴드 공연 볼 예정이야. 민준은 요즘 라이브 음악에 빠졌거든."},
    outputs={"ai": "좋다 민준아! 나는 같은 날 오후 3시에 성수에서 카페 투어 할 계획이야. 새로운 스페셜티 카페를 지도에 정리해 두었어."},
)

memory.save_context(
    inputs={"human": "민준은 드립 커피를 좋아하고, 특히 에티오피아 내추럴을 선호해. 지혜는 라떼파이브의 플랫화이트가 최애 메뉴지?"},
    outputs={"ai": "맞아, 플랫화이트를 가장 좋아해. 대신 저녁에는 카페인 줄이려고 디카페인으로 바꿔 마셔."},
)

memory.save_context(
    inputs={"human": "우리 사이드 프로젝트 ‘플레이리스트 지도’는 이번 주 일요일(8월 24일) 자정에 MVP 마감이야."},
    outputs={"ai": "일정 확인! 나는 프런트엔드 페이지 라우팅과 검색 UI를 끝내고, 남은 시간엔 반응형 레이아웃을 다듬을게."},
)

memory.save_context(
    inputs={"human": "민준은 백엔드에서 FastAPI로 트랙 추천 API를 만들고 있어. 벡터 검색은 FAISS로, 임베딩은 all-MiniLM-L6-v2를 쓸 예정이야."},
    outputs={"ai": "좋아, 그럼 검색창에서 태그+가수 이름을 함께 넣어도 의미 기반으로 잘 매칭되겠네."},
)

memory.save_context(
    inputs={"human": "데이터 출처는 멜론 플레이리스트 크롤링과 스포티파이 공개 API야. 저작권 표기는 푸터에 명시할게."},
    outputs={"ai": "라이선스 표기 체크 완료. 개인정보는 수집하지 않으니 개인정보 처리방침은 최소한으로 작성해도 되겠다."},
)

memory.save_context(
    inputs={"human": "배포는 Vercel(프런트)과 Railway(백엔드)로 갈래? 비용은 무료 티어를 우선 쓰고, 트래픽 늘면 업그레이드 하자."},
    outputs={"ai": "동의! 도메인은 playmap.dev 후보로 두고, 우선 서브도메인으로 시작하자."},
)

memory.save_context(
    inputs={"human": "연락은 급하면 슬랙 #mvp 채널로, 일반 문의는 지메일로 받자. 데모 영상은 8월 24일 저녁 8시에 리허설할게."},
    outputs={"ai": "오케이, 리허설 후 체크리스트를 노션에 정리해서 공유할게. 이슈는 깃허브 프로젝트 보드로 관리하자."},
)

memory.save_context(
    inputs={"human": "참, 공연 끝나고 9시 반쯤 홍대 입구역에서 합류할게. 지혜는 성수 연무장길 카페에서 8시까지 있다고 했지?"},
    outputs={"ai": "맞아, 8시에 성수 쪽 일정 마치고 이동할게. 혹시 늦어지면 카카오톡으로 위치 공유할게."},
)


다음의 질문을 했을 때 Vector Store 로 부터 1개(k=1 이기 때문)의 가장 관련성 높은 대화를 반환합니다.

- 질문: "면접자 전공은 무엇인가요?"


In [11]:
# 메모리에 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
print(memory.load_memory_variables({"human": "MVP 마감 기한은 언제였지?"})["history"])

human: 우리 사이드 프로젝트 ‘플레이리스트 지도’는 이번 주 일요일(8월 24일) 자정에 MVP 마감이야.
ai: 일정 확인! 나는 프런트엔드 페이지 라우팅과 검색 UI를 끝내고, 남은 시간엔 반응형 레이아웃을 다듬을게.


이번에는 다른 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.

- 질문: "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"


In [12]:
print(
    memory.load_memory_variables(
        {"human": "민준이 사용하는 벡터 검색 도구는 뭐야?"}
    )["history"]
)

human: 민준은 백엔드에서 FastAPI로 트랙 추천 API를 만들고 있어. 벡터 검색은 FAISS로, 임베딩은 all-MiniLM-L6-v2를 쓸 예정이야.
ai: 좋아, 그럼 검색창에서 태그+가수 이름을 함께 넣어도 의미 기반으로 잘 매칭되겠네.
